# Phi-4 Mini ONNX Parallel Function Calling Tutorial

এই নোটবুকটি Phi-4 Mini এবং ONNX Runtime GenAI ব্যবহার করে সমান্তরাল ফাংশন কলিং কীভাবে করা যায় তা প্রদর্শন করে। ফাংশন কলিং মডেলকে ব্যবহারকারীর অনুরোধ অনুযায়ী বুদ্ধিমত্তার সাথে বাহ্যিক টুল এবং API-গুলো আহ্বান করতে সক্ষম করে।

## সংক্ষিপ্ত বিবরণ

এই টিউটোরিয়ালে আপনি শিখবেন:
- Phi-4 Mini এবং ONNX Runtime GenAI সেট আপ করা
- ফ্লাইট এবং হোটেল বুকিংয়ের জন্য ফাংশন স্কিমা সংজ্ঞায়িত করা
- কাঠামোগত আউটপুটের জন্য Lark ব্যাকরণ সহ নির্দেশিত জেনারেশন ব্যবহার করা
- জটিল ভ্রমণ বুকিং পরিস্থিতির জন্য সমান্তরাল ফাংশন কল সম্পাদন করা

## প্রয়োজনীয়তা

এই নোটবুক চালানোর আগে নিশ্চিত করুন যে আপনার কাছে রয়েছে:
- Phi-4 Mini ONNX মডেল ডাউনলোড করা
- `onnxruntime-genai` প্যাকেজ ইনস্টল করা
- ফাংশন কলিং ধারণার মৌলিক ধারণা


## ধাপ ১: প্রয়োজনীয় লাইব্রেরি আমদানি করুন

প্রথমে, আমরা আমাদের ফাংশন কলিং বাস্তবায়নের জন্য প্রয়োজনীয় লাইব্রেরিগুলি আমদানি করব।


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## ধাপ ২: মডেল সেটআপ এবং কনফিগারেশন

এখন আমরা Phi-4 Mini ONNX মডেলটি কনফিগার করব। আপনার প্রকৃত মডেল ডিরেক্টরির পথটি এখানে প্রতিস্থাপন করতে ভুলবেন না।


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## ধাপ ৩: জেনারেশন প্যারামিটার কনফিগার করুন

মডেলের আউটপুট আচরণ নিয়ন্ত্রণ করতে জেনারেশন প্যারামিটার সেট আপ করুন। এই সেটিংসগুলি ফাংশন কলিংয়ের জন্য নির্ধারিত এবং কেন্দ্রীভূত প্রতিক্রিয়া নিশ্চিত করে।


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## ধাপ ৪: উপলব্ধ ফাংশনগুলি সংজ্ঞায়িত করুন

এখানে আমরা সেই ফাংশনগুলি সংজ্ঞায়িত করছি যা আমাদের AI সহকারী কল করতে পারে। এই উদাহরণে, আমাদের কাছে দুটি ভ্রমণ-সম্পর্কিত ফাংশন রয়েছে:
1. **booking_flight_tickets**: বিমানবন্দরের মধ্যে ফ্লাইট বুক করার জন্য
2. **booking_hotels**: হোটেল থাকার ব্যবস্থা বুক করার জন্য

ফাংশন সংজ্ঞাগুলি OpenAI-এর ফাংশন কলিং স্কিমা ফরম্যাট অনুসরণ করে।


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## ধাপ ৫: ব্যাকরণ তৈরির সহায়ক ফাংশনসমূহ

এই সহায়ক ফাংশনগুলো আমাদের ফাংশন সংজ্ঞাগুলোকে Lark ব্যাকরণ ফরম্যাটে রূপান্তর করে, যা ONNX Runtime GenAI দ্বারা নির্দেশিত জেনারেশনের জন্য ব্যবহৃত হয়। এটি নিশ্চিত করে যে মডেল সঠিক JSON ফরম্যাটে বৈধ ফাংশন কল আউটপুট করে।


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## ধাপ ৬: ব্যাকরণ তৈরির পরীক্ষা

চলুন আমাদের ব্যাকরণ তৈরির ফাংশনগুলো পরীক্ষা করি এবং দেখি কীভাবে তারা আমাদের টুল সংজ্ঞাগুলোকে সঠিক ফরম্যাটে রূপান্তর করে।


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## ধাপ ৭: সিস্টেম প্রম্পট এবং জেনারেটর প্রস্তুত করুন

এখন আমরা সিস্টেম প্রম্পট তৈরি করব যা মডেলকে উপলব্ধ টুলগুলোর সম্পর্কে জানাবে এবং নির্দেশিত জেনারেশন প্যারামিটার দিয়ে জেনারেটর সেট আপ করব।


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## ধাপ ৮: নির্দেশিত জেনারেশনের জন্য জেনারেটর আরম্ভ করুন

এখন আমরা আমাদের কনফিগার করা প্যারামিটারগুলির সাথে জেনারেটর তৈরি করব এবং নির্দেশিত জেনারেশনের জন্য লার্ক ব্যাকরণ প্রয়োগ করব।


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## ধাপ ৯: সমান্তরাল ফাংশন কলিং পরীক্ষা করুন

এখন চলুন একটি জটিল ভ্রমণ বুকিং পরিস্থিতি পরীক্ষা করি, যেখানে একাধিক ফাংশন কল করতে হবে।


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### ফাংশন কল তৈরি করুন

মডেল এখন আমাদের ব্যবহারকারীর অনুরোধ অনুযায়ী কাঠামোবদ্ধ ফাংশন কল তৈরি করবে। নির্দেশিত জেনারেশনের কারণে, আউটপুট বৈধ JSON ফরম্যাটে থাকবে যা সরাসরি কার্যকর করা যেতে পারে।

**প্রত্যাশিত আউটপুট**: মডেলটি নিম্নলিখিত ফাংশন কল তৈরি করবে:
1. `booking_flight_tickets` এর জন্য বেইজিং (PEK) থেকে প্যারিস (CDG) এর বিবরণ সহ
2. `booking_hotels` এর জন্য প্যারিসে থাকার বিবরণ সহ

নীচের সেলটি চালান লাইভ জেনারেশন দেখতে:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## উপসংহার

🎉 **অভিনন্দন!** আপনি সফলভাবে Phi-4 Mini ব্যবহার করে ONNX Runtime GenAI এর মাধ্যমে সমান্তরাল ফাংশন কলিং বাস্তবায়ন করেছেন।

### আপনি যা শিখেছেন:

1. **মডেল সেটআপ**: ONNX Runtime GenAI এর সাথে Phi-4 Mini কনফিগার করার পদ্ধতি
2. **ফাংশন সংজ্ঞা**: AI ফাংশন কলিংয়ের জন্য ফাংশন স্কিমা কীভাবে সংজ্ঞায়িত করবেন
3. **নির্দেশিত জেনারেশন**: কাঠামোবদ্ধ আউটপুট তৈরির জন্য Lark ব্যাকরণ ব্যবহার করার পদ্ধতি
4. **সমান্তরাল ফাংশন কল**: জটিল অনুরোধ পরিচালনা করার জন্য একাধিক ফাংশন কলের ব্যবহার

### প্রধান সুবিধাসমূহ:

- ✅ **কাঠামোবদ্ধ আউটপুট**: নির্দেশিত জেনারেশন বৈধ JSON ফাংশন কল নিশ্চিত করে
- ✅ **সমান্তরাল প্রসেসিং**: একক অনুরোধে একাধিক ফাংশন কল পরিচালনা করা যায়
- ✅ **উচ্চ কার্যক্ষমতা**: ONNX Runtime অপ্টিমাইজড ইনফারেন্স প্রদান করে
- ✅ **লচনীয় স্কিমা**: ফাংশন সংজ্ঞা সহজে যোগ বা পরিবর্তন করা যায়

### সম্পদসমূহ:

- [Phi-4 Mini ডকুমেন্টেশন](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI ডকুমেন্টেশন](https://onnxruntime.ai/docs/genai/)
- [ফাংশন কলিংয়ের সেরা অনুশীলন](https://platform.openai.com/docs/guides/function-calling)



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
